In [2]:
import random
import pandas as pd
import numpy as np

# Creation of the tile class
This class will be used to represent a cell in the grid and is responsible to hold information of the available options, if the cell has collapsed already and the value chosen iof applicable

In [103]:
class Tile:
    options = []
    collapsed = False
    value = None
    position = ()

    def __init__(self, y, x):
        self.options = [i for i in range(1, 10)] # set options for a cell from 1 to 9
        self.collapsed = False
        self.value = None
        self.position = (y, x)

    def setValue(self, v):
        self.value = v
        self.collapsed = True

    def getEnthropy(self):
        return len(self.options)

    def __str__(self):
        return f'tile {self.position} \n' \
               f'options : {[i for i in self.options]} \n' \
               f'value : {self.value} \n' \
               f'collapsed : {self.collapsed} \n'

    def __le__(self, other):
        return self.getEnthropy() < other.getEnthropy()

    def __eq__(self, other):
        return self.getEnthropy() == other.getEnthropy()

    def __gt__(self, other):
        return self.getEnthropy() > other.getEnthropy()


# Creation of the grid

In [104]:
grid = np.empty(shape=(9,9), dtype=Tile)
for y in range(9):
    for x in range(9):
        grid[y, x] = Tile(y, x)


# System to display the sudoku grid

In [105]:
def displayGrid(target):
    display = ""
    for y in range(9):
        for x in range(9):
            display += (" " + str(target[y, x].value) + " ")
        display += "\n"
    print(display)


In [106]:
def validateGrid(target):
    for i in range(9):
        try:
            row_sum = sum([t.value for t in grid[i, :]])
            col_sum = sum([t.value for t in grid[:, i]])
        except TypeError:
            return False
        if row_sum != 45 or col_sum != 45:
            return False
    return True


In [107]:
displayGrid(grid)
validateGrid(grid)

 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 
 None  None  None  None  None  None  None  None  None 



False

# Algorithm
We create functions for the algorithm
- Function to select cell with lowest enthropy
- Function to collapse a cell
- Function to propagate changes

In [156]:
def getMinEnthropyCell(target):
    pass

# set an arbitrary value to collapse 0,0
grid[0,0].setValue(1)
collapsed_filter = np.array([t.collapsed for t in grid.ravel()])
grid.ravel()[~collapsed_filter]

array([<__main__.Tile object at 0x000002A880AF8D90>,
       <__main__.Tile object at 0x000002A880AF9810>], dtype=object)